In [72]:

import pandas as pd
import plotly.express as px
pd.options.display.max_colwidth = 200

from Project.Database import Db
from Project._05InferKnowledgeOfRules.infer_rules_functions import json_to_dataframe, SE_time_df, redundancy_filter_tool

# Phase 1
Define available appliances

In [73]:
meta = Db.load_data(meta=True, hourly=False, consumption=False)
appliance_list = meta.loc[(lambda self:
                           (self["Parameter"] == "Status_OnOff") &
                           (~ self.index.str.contains("SensHeat")) &
                           (~ self.index.str.contains("Cold")))].index.to_list()
appliance_list

['DHW_StatusSolenoidHotKitchenSink',
 'DHW_StatusSolenoidHotMBAShower',
 'DHW_StatusSolenoidHotMBATub',
 'Load_StatusApplianceCooktop',
 'Load_StatusApplianceDishwasher',
 'Load_StatusApplianceOven',
 'Load_StatusApplianceRangeHood',
 'Load_StatusBA1Lights',
 'Load_StatusBA2Lights',
 'Load_StatusBR2Lights',
 'Load_StatusBR3Lights',
 'Load_StatusBR4Lights',
 'Load_StatusDRLights',
 'Load_StatusEntryHallLights',
 'Load_StatusKitchenLightsA',
 'Load_StatusKitchenLightsB',
 'Load_StatusKitchenLightsC',
 'Load_StatusLRLights1',
 'Load_StatusLRLights2',
 'Load_StatusLRLights3',
 'Load_StatusLatentload',
 'Load_StatusMBALights',
 'Load_StatusMBRLights1',
 'Load_StatusMBRLights2',
 'Load_StatusPlugLoadBR2Laptop',
 'Load_StatusPlugLoadBR3Laptop',
 'Load_StatusPlugLoadBlender',
 'Load_StatusPlugLoadCanOpener',
 'Load_StatusPlugLoadCoffeeMaker',
 'Load_StatusPlugLoadDesktopPCMonitor',
 'Load_StatusPlugLoadFan',
 'Load_StatusPlugLoadHairDryerCurlIron',
 'Load_StatusPlugLoadHandMixer',
 'Load_Statu

In [74]:
meta[['Load_Match', 'Elec_Match', 'DHW_Match']]

,Load_Match,Elec_Match,DHW_Match
Unnamed: 0,,,
DHW_ClothesWasherColdFlow,NaN,NaN,NaN
DHW_ClothesWasherHotFlow,NaN,NaN,NaN
DHW_DishwasherHotFlow,NaN,NaN,NaN
DHW_HeatPumpWaterHeaterEnergyTotal,NaN,NaN,NaN
DHW_HeatPumpWaterHeaterPowerTotal,NaN,NaN,NaN
...,...,...,...
HVAC_HeatPumpIndoorUnitPower,NaN,NaN,NaN
HVAC_HeatPumpOutdoorUnitPower,NaN,NaN,NaN
Load_StatusClothesWasher,Load_ClothesWasherPowerWithStandby,Elec_PowerClothesWasher,NaN


# Phase 2
Find constraints of devices:
o	If there are hard constraints for this appliance only
o	Does it have to be on in a continous period of time? How long does it last?
o	How many watts pr second does it use?
o	How much energy does it need from start to finish?


## Time duration constraints

In [75]:
level1_rules_year1 = json_to_dataframe(year=1, level=1)
level1_rules_year1

,pattern,supp,conf,time,rule,multi_floor,floor
0,Load_StatusMBALights,0.991781,1.0,"{'0': [['2013-07-01 03:29:17', '2013-07-01 03:32:17'], ['2013-07-01 03:58:17', '2013-07-01 04:01:17'], ['2013-07-01 05:58:17', '2013-07-01 06:13:17'], ['2013-07-01 06:28:17', '2013-07-01 06:43:17'...",app,False,2ndFloor
1,Load_StatusSensHeatPrntBDOWN,0.991781,1.0,"{'0': [['2013-07-01 05:58:17', '2013-07-01 06:28:17'], ['2013-07-01 06:58:17', '2013-07-01 08:28:17'], ['2013-07-01 17:58:31', '2013-07-01 21:58:30']], '1': [['2013-07-02 05:58:29', '2013-07-02 06...",psn,False,1stFloor
2,Load_StatusKitchenLights,0.991781,1.0,"{'0': [['2013-07-01 05:58:17', '2013-07-01 08:23:17'], ['2013-07-01 17:58:31', '2013-07-01 18:28:31'], ['2013-07-01 18:58:31', '2013-07-01 19:13:31'], ['2013-07-01 19:28:31', '2013-07-01 20:13:31'...",app,False,1stFloor
3,Load_StatusPlugLoadBlender,0.980822,1.0,"{'0': [['2013-07-01 06:03:17', '2013-07-01 06:10:17']], '1': [['2013-07-02 06:03:29', '2013-07-02 06:10:29']], '2': [['2013-07-03 06:03:28', '2013-07-03 06:10:28']], '3': [['2013-07-04 06:03:27', ...",app,False,1stFloor
4,Load_StatusPlugLoadCoffeeMaker,0.980822,1.0,"{'0': [['2013-07-01 06:08:17', '2013-07-01 06:26:17']], '1': [['2013-07-02 06:08:29', '2013-07-02 06:38:29']], '2': [['2013-07-03 06:08:28', '2013-07-03 06:38:28']], '3': [['2013-07-04 06:08:27', ...",app,False,1stFloor
5,Load_StatusPlugLoadMBRTV,0.843836,1.0,"{'0': [['2013-07-01 06:13:17', '2013-07-01 06:58:17']], '1': [['2013-07-02 06:13:29', '2013-07-02 06:58:29']], '2': [['2013-07-03 06:13:28', '2013-07-03 06:58:28'], ['2013-07-03 21:58:27', '2013-0...",app,False,2ndFloor
6,Load_StatusPlugLoadToasterOven,0.980822,1.0,"{'0': [['2013-07-01 06:13:17', '2013-07-01 06:23:17']], '1': [['2013-07-02 06:13:29', '2013-07-02 06:23:29']], '2': [['2013-07-03 06:13:28', '2013-07-03 06:23:28']], '3': [['2013-07-04 06:13:27', ...",app,False,1stFloor
7,Load_StatusMasterBedroomLights,0.991781,1.0,"{'0': [['2013-07-01 06:13:17', '2013-07-01 06:28:17'], ['2013-07-01 06:43:17', '2013-07-01 06:58:17'], ['2013-07-01 18:05:31', '2013-07-01 18:28:31'], ['2013-07-01 21:58:30', '2013-07-01 22:58:30'...",app,False,2ndFloor
8,Load_StatusPlugLoadToaster,0.980822,1.0,"{'0': [['2013-07-01 06:18:17', '2013-07-01 06:29:17']], '1': [['2013-07-02 06:18:29', '2013-07-02 06:29:29']], '2': [['2013-07-03 06:18:28', '2013-07-03 06:29:28']], '3': [['2013-07-04 06:18:27', ...",app,False,1stFloor
9,Load_StatusBA2Lights,0.991781,1.0,"{'0': [['2013-07-01 06:28:17', '2013-07-01 06:48:17'], ['2013-07-01 18:06:31', '2013-07-01 18:16:31'], ['2013-07-01 20:13:31', '2013-07-01 20:23:31'], ['2013-07-01 21:43:30', '2013-07-01 21:58:30'...",app,False,2ndFloor


In [76]:
def find_intervals():
    time_delta_dict = {}
    for year in [1, 2]:
        time_delta_dict[f'year{year}'] = time_delta_dict.get(f'year{year}', {})
        for index, row in json_to_dataframe(year=year, level=1).iterrows():
            if 'SensHeat' not in row['pattern']:
                time_delta_dict[f'year{year}'][row["pattern"]] = pd.Series(dtype='float64')
                for intervals in row["time"].values():
                   for interval in intervals:
                       time_delta_dict[f'year{year}'][row["pattern"]] = pd.concat((time_delta_dict[f'year{year}'][row["pattern"]], pd.Series((pd.to_datetime(interval[1], format="%Y-%m-%d %H:%M:%S") - pd.to_datetime(interval[0], format="%Y-%m-%d %H:%M:%S")).total_seconds()/60)), ignore_index=True)
    return time_delta_dict

In [77]:
time_delta_dict = find_intervals()

In [78]:
def describe_intervals(att):
    if att not in time_delta_dict['year1'].keys():
        return time_delta_dict['year2'][att].describe()
    elif att not in time_delta_dict['year2'].keys():
        return time_delta_dict['year1'][att].describe()
    else:
        return pd.concat((time_delta_dict['year1'][att], time_delta_dict['year2'][att]), axis=0).describe()


apps = set(list(time_delta_dict['year1'].keys()) + list(set(time_delta_dict['year2'].keys())))

for pattern in apps:
    if 'SensHeat' not in pattern:
        print(pattern, describe_intervals(pattern))

Load_StatusBA1Lights count    1345.000000
mean        4.883494
std        13.267184
min         3.266667
25%         4.000000
50%         4.000000
75%         5.000000
max       467.266667
dtype: float64
Load_StatusPlugLoadIron count    208.000000
mean      23.782933
std        1.409483
min        8.000000
25%       24.000000
50%       24.000000
75%       24.000000
max       25.000000
dtype: float64
Load_StatusMBALights count    4532.000000
mean       10.959852
std         6.144704
min         2.833333
25%         5.000000
50%        15.000000
75%        15.000000
max       113.000000
dtype: float64
Load_StatusLivingRoomLights count    2470.000000
mean       70.075884
std        79.537204
min        14.983333
25%        25.000000
50%        30.000000
75%       120.000000
max      1085.433333
dtype: float64
Load_StatusPlugLoadCoffeeMaker count    760.000000
mean      18.098750
std        1.429868
min       17.000000
25%       18.000000
50%       18.000000
75%       18.000000
max       3

In [79]:
time_delta_dict['year1']['Load_StatusClothesWasher'][lambda self: self <130]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
306    57.0
307    56.0
308    57.0
309    61.0
310    57.0
Length: 311, dtype: float64

In [80]:
def find_quantile(quantile, att):
    return time_delta_dict['year1'][att].quantile(quantile), time_delta_dict['year2'][att].quantile(quantile)
find_quantile(0.90, "Load_StatusMBALights")

(20.0, 20.0)

In [81]:
hist = pd.Series(time_delta_dict['year1']["Load_StatusApplianceDishwasher"])
fig = px.histogram(hist)
fig.show()

In [82]:

fig = px.box(time_delta_dict['year1'], log_y=True)
fig.update_traces(jitter= 0.3)
# fig.show()
fig.write_html(Db.get_save_file_directory("rule_constraints/time_delta_boxplot.html"))

## Temporal associations constraints (Which hours is the appliances associated with)

In [83]:
SE_time_df(level1_rules_year1)

{'Load_StatusMBALights':     TotalAbsSupport  AbsSupport  EventCount  ExternalUtility  RelSupport  \
 0                 0           0        2265         0.991781    0.000000   
 1                 1           1        2265         0.991781    0.002740   
 2                 0           0        2265         0.991781    0.000000   
 3               495         258        2265         0.991781    0.706849   
 4               258         258        2265         0.991781    0.706849   
 5               238         238        2265         0.991781    0.652055   
 6               514         257        2265         0.991781    0.704110   
 7                96          96        2265         0.991781    0.263014   
 8               101         101        2265         0.991781    0.276712   
 9                95          95        2265         0.991781    0.260274   
 10              101         101        2265         0.991781    0.276712   
 11                2           1        2265        

## Association dependency (how dependent is an appliance on another appliance)

In [84]:
level2_rules_year1 = json_to_dataframe(year=1, level=2, exclude_follows=False)

In [85]:
level2_rules_year1

,pattern,supp,conf,time,rule,multi_floor,floor
0,Load_StatusMBALights->Load_StatusMBALights,0.849315,0.856354,"{'41': [[['2013-08-11 22:04:05', '2013-08-11 22:24:05'], ['2013-08-11 22:29:05', '2013-08-11 22:39:04']]], '144': [[['2013-11-22 03:30:09', '2013-11-22 03:33:09'], ['2013-11-22 03:59:09', '2013-11...",app_app,False,2ndFloor
1,Load_StatusMBALights->Load_StatusSensHeatPrntBDOWN,0.980822,0.988950,"{'41': [[['2013-08-11 07:59:06', '2013-08-11 08:14:06'], ['2013-08-11 08:14:06', '2013-08-11 09:59:06']]], '144': [[['2013-11-22 06:29:09', '2013-11-22 06:44:09'], ['2013-11-22 06:59:09', '2013-11...",psn_app,True,multi
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 05:59:09', '2013-11-22 06:29:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 05:59:04', '2013-1...",psn_app,True,multi
3,Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 06:04:09', '2013-11-22 06:10:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 06:04:04', '2013-1...",app_app,True,multi
4,Load_StatusMBALights>Load_StatusPlugLoadCoffeeMaker,0.701370,0.707182,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 06:09:09', '2013-11-22 06:26:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 06:09:04', '2013-1...",app_app,True,multi
...,...,...,...,...,...,...,...
624,Load_StatusApplianceOven->Load_StatusClothesWasher,0.276712,0.651613,"{'194': [[['2014-01-11 08:14:15', '2014-01-11 10:39:15'], ['2014-01-11 11:00:15', '2014-01-11 12:00:15']]], '41': [[['2013-08-11 08:14:06', '2013-08-11 10:49:05'], ['2013-08-11 11:00:05', '2013-08...",app_app,False,1stFloor
625,Load_StatusClothesWasher>Load_StatusDryerPowerTotal,0.405479,0.896970,"{'194': [[['2014-01-11 14:30:15', '2014-01-11 15:38:15'], ['2014-01-11 15:00:15', '2014-01-11 15:41:15']]], '205': [[['2014-01-22 20:25:53', '2014-01-22 21:25:53'], ['2014-01-22 20:30:53', '2014-0...",app_app,False,1stFloor
626,Load_StatusClothesWasher|Load_StatusPlugLoadLRBlueRay,0.268493,0.632258,"{'194': [[['2014-01-11 14:30:15', '2014-01-11 15:38:15'], ['2014-01-11 14:59:15', '2014-01-11 16:29:14']]], '41': [[['2013-08-11 14:30:05', '2013-08-11 15:27:05'], ['2013-08-11 14:59:05', '2013-08...",app_app,False,1stFloor
627,Load_StatusPlugLoadLRBlueRay>Load_StatusDryerPowerTotal,0.279452,0.618182,"{'194': [[['2014-01-11 14:59:15', '2014-01-11 16:29:14'], ['2014-01-11 15:00:15', '2014-01-11 15:41:15']]], '41': [[['2013-08-11 14:59:05', '2013-08-11 16:29:05'], ['2013-08-11 14:59:05', '2013-08...",app_app,False,1stFloor


In [86]:
appended_data = []
for i in [True, False]:
    data = redundancy_filter_tool(level2_rules_year1, regex_str='', multi_floor=i, rule_type='app_app')
    # store DataFrame in list
    appended_data.append(data)
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
appended_data

,pattern,supp,conf,time,rule,multi_floor,floor
3,Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 06:04:09', '2013-11-22 06:10:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 06:04:04', '2013-1...",app_app,True,multi
4,Load_StatusMBALights>Load_StatusPlugLoadCoffeeMaker,0.701370,0.707182,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 06:09:09', '2013-11-22 06:26:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 06:09:04', '2013-1...",app_app,True,multi
6,Load_StatusMBALights->Load_StatusPlugLoadToasterOven,0.701370,0.707182,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 06:14:09', '2013-11-22 06:24:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 06:14:04', '2013-1...",app_app,True,multi
8,Load_StatusMBALights->Load_StatusPlugLoadToaster,0.701370,0.707182,"{'144': [[['2013-11-22 05:59:09', '2013-11-22 06:14:09'], ['2013-11-22 06:19:09', '2013-11-22 06:30:09']]], '135': [[['2013-11-13 05:59:04', '2013-11-13 06:14:04'], ['2013-11-13 06:19:04', '2013-1...",app_app,True,multi
11,Load_StatusMBALights->Load_StatusPlugLoadLRTV,0.986301,0.994475,"{'41': [[['2013-08-11 07:59:06', '2013-08-11 08:14:06'], ['2013-08-11 08:29:06', '2013-08-11 10:29:05']], [['2013-08-11 18:00:05', '2013-08-11 18:05:05'], ['2013-08-11 18:09:05', '2013-08-11 22:39...",app_app,True,multi
...,...,...,...,...,...,...,...
623,Load_StatusApplianceOven>Load_StatusPlugLoadVideoGame,0.279452,0.666667,"{'194': [[['2014-01-11 08:14:15', '2014-01-11 10:39:15'], ['2014-01-11 09:59:15', '2014-01-11 10:59:15']]], '41': [[['2013-08-11 08:14:06', '2013-08-11 10:49:05'], ['2013-08-11 09:59:06', '2013-08...",app_app,False,1stFloor
624,Load_StatusApplianceOven->Load_StatusClothesWasher,0.276712,0.651613,"{'194': [[['2014-01-11 08:14:15', '2014-01-11 10:39:15'], ['2014-01-11 11:00:15', '2014-01-11 12:00:15']]], '41': [[['2013-08-11 08:14:06', '2013-08-11 10:49:05'], ['2013-08-11 11:00:05', '2013-08...",app_app,False,1stFloor
625,Load_StatusClothesWasher>Load_StatusDryerPowerTotal,0.405479,0.896970,"{'194': [[['2014-01-11 14:30:15', '2014-01-11 15:38:15'], ['2014-01-11 15:00:15', '2014-01-11 15:41:15']]], '205': [[['2014-01-22 20:25:53', '2014-01-22 21:25:53'], ['2014-01-22 20:30:53', '2014-0...",app_app,False,1stFloor
626,Load_StatusClothesWasher|Load_StatusPlugLoadLRBlueRay,0.268493,0.632258,"{'194': [[['2014-01-11 14:30:15', '2014-01-11 15:38:15'], ['2014-01-11 14:59:15', '2014-01-11 16:29:14']]], '41': [[['2013-08-11 14:30:05', '2013-08-11 15:27:05'], ['2013-08-11 14:59:05', '2013-08...",app_app,False,1stFloor


In [87]:
appended_data.loc[lambda self: (self['conf'] > 0.99)]

,pattern,supp,conf,time,rule,multi_floor,floor
11,Load_StatusMBALights->Load_StatusPlugLoadLRTV,0.986301,0.994475,"{'41': [[['2013-08-11 07:59:06', '2013-08-11 08:14:06'], ['2013-08-11 08:29:06', '2013-08-11 10:29:05']], [['2013-08-11 18:00:05', '2013-08-11 18:05:05'], ['2013-08-11 18:09:05', '2013-08-11 22:39...",app_app,True,multi
30,Load_StatusMBALights->Load_StatusPlugLoadDesktopPCMonitor,0.983562,0.991713,"{'41': [[['2013-08-11 07:59:06', '2013-08-11 08:14:06'], ['2013-08-11 08:29:06', '2013-08-11 18:59:05']]], '144': [[['2013-11-22 18:00:08', '2013-11-22 18:05:08'], ['2013-11-22 18:29:08', '2013-11...",app_app,True,multi
64,Load_StatusKitchenLights->Load_StatusMBALights,0.986301,0.994475,"{'41': [[['2013-08-11 08:14:06', '2013-08-11 09:59:06'], ['2013-08-11 09:59:06', '2013-08-11 10:14:05']], [['2013-08-11 20:29:05', '2013-08-11 22:29:05'], ['2013-08-11 22:29:05', '2013-08-11 22:39...",app_app,True,multi
65,Load_StatusKitchenLights>Load_StatusMBALights,0.991781,1.000000,"{'41': [[['2013-08-11 17:59:05', '2013-08-11 18:29:05'], ['2013-08-11 18:00:05', '2013-08-11 18:05:05']], [['2013-08-11 20:29:05', '2013-08-11 22:29:05'], ['2013-08-11 22:04:05', '2013-08-11 22:24...",app_app,True,multi
72,Load_StatusKitchenLights>Load_StatusMasterBedroomLights,0.989041,0.997238,"{'41': [[['2013-08-11 17:59:05', '2013-08-11 18:29:05'], ['2013-08-11 18:06:05', '2013-08-11 18:29:05']], [['2013-08-11 20:29:05', '2013-08-11 22:29:05'], ['2013-08-11 21:59:05', '2013-08-11 22:59...",app_app,True,multi
75,Load_StatusKitchenLights>Load_StatusBA2Lights,0.991781,1.000000,"{'41': [[['2013-08-11 08:14:06', '2013-08-11 09:59:06'], ['2013-08-11 08:44:06', '2013-08-11 09:14:06']], [['2013-08-11 17:59:05', '2013-08-11 18:29:05'], ['2013-08-11 18:07:05', '2013-08-11 18:17...",app_app,True,multi
190,Load_StatusMasterBedroomLights->Load_StatusPlugLoadCanOpener,0.986301,0.994475,"{'41': [[['2013-08-11 18:06:05', '2013-08-11 18:29:05'], ['2013-08-11 18:39:05', '2013-08-11 18:46:05']]], '144': [[['2013-11-22 18:06:08', '2013-11-22 18:29:08'], ['2013-11-22 18:39:08', '2013-11...",app_app,True,multi
191,Load_StatusMasterBedroomLights->Load_StatusPlugLoadHandMixer,0.986301,0.994475,"{'41': [[['2013-08-11 18:06:05', '2013-08-11 18:29:05'], ['2013-08-11 18:39:05', '2013-08-11 18:41:05']]], '144': [[['2013-11-22 18:06:08', '2013-11-22 18:29:08'], ['2013-11-22 18:39:08', '2013-11...",app_app,True,multi
236,Load_StatusPlugLoadLRTV>Load_StatusBA2Lights,0.986301,0.994475,"{'41': [[['2013-08-11 08:29:06', '2013-08-11 10:29:05'], ['2013-08-11 08:44:06', '2013-08-11 09:14:06']], [['2013-08-11 18:09:05', '2013-08-11 22:39:04'], ['2013-08-11 20:14:05', '2013-08-11 20:24...",app_app,True,multi
320,Load_StatusBR2Lights>Load_StatusPlugLoadCanOpener,0.989041,0.997238,"{'41': [[['2013-08-11 18:19:05', '2013-08-12 00:00:04'], ['2013-08-11 18:39:05', '2013-08-11 18:46:05']]], '144': [[['2013-11-22 18:19:08', '2013-11-22 20:59:08'], ['2013-11-22 18:39:08', '2013-11...",app_app,True,multi


In [88]:
appended_data.loc[lambda self: (self['conf'] == 1)]

,pattern,supp,conf,time,rule,multi_floor,floor
65,Load_StatusKitchenLights>Load_StatusMBALights,0.991781,1.0,"{'41': [[['2013-08-11 17:59:05', '2013-08-11 18:29:05'], ['2013-08-11 18:00:05', '2013-08-11 18:05:05']], [['2013-08-11 20:29:05', '2013-08-11 22:29:05'], ['2013-08-11 22:04:05', '2013-08-11 22:24...",app_app,True,multi
75,Load_StatusKitchenLights>Load_StatusBA2Lights,0.991781,1.0,"{'41': [[['2013-08-11 08:14:06', '2013-08-11 09:59:06'], ['2013-08-11 08:44:06', '2013-08-11 09:14:06']], [['2013-08-11 17:59:05', '2013-08-11 18:29:05'], ['2013-08-11 18:07:05', '2013-08-11 18:17...",app_app,True,multi
558,Load_StatusPlugLoadBR3Laptop>Load_StatusPlugLoadCanOpener,0.989041,1.0,"{'41': [[['2013-08-11 18:29:05', '2013-08-11 20:14:05'], ['2013-08-11 18:39:05', '2013-08-11 18:46:05']]], '144': [[['2013-11-22 18:29:08', '2013-11-22 20:59:08'], ['2013-11-22 18:39:08', '2013-11...",app_app,True,multi
559,Load_StatusPlugLoadBR3Laptop>Load_StatusPlugLoadHandMixer,0.989041,1.0,"{'41': [[['2013-08-11 18:29:05', '2013-08-11 20:14:05'], ['2013-08-11 18:39:05', '2013-08-11 18:41:05']]], '144': [[['2013-11-22 18:29:08', '2013-11-22 20:59:08'], ['2013-11-22 18:39:08', '2013-11...",app_app,True,multi
7,Load_StatusMBALights->Load_StatusMasterBedroomLights,0.991781,1.0,"{'41': [[['2013-08-11 09:59:06', '2013-08-11 10:14:05'], ['2013-08-11 10:14:05', '2013-08-11 10:59:05']], [['2013-08-11 18:00:05', '2013-08-11 18:05:05'], ['2013-08-11 18:06:05', '2013-08-11 18:29...",app_app,False,2ndFloor
124,Load_StatusPlugLoadCoffeeMaker>Load_StatusPlugLoadToasterOven,0.980822,1.0,"{'41': [[['2013-08-11 09:09:06', '2013-08-11 09:27:06'], ['2013-08-11 09:19:06', '2013-08-11 09:29:06']]], '144': [[['2013-11-22 06:09:09', '2013-11-22 06:26:09'], ['2013-11-22 06:14:09', '2013-11...",app_app,False,1stFloor
127,Load_StatusPlugLoadCoffeeMaker>Load_StatusPlugLoadToaster,0.980822,1.0,"{'41': [[['2013-08-11 09:09:06', '2013-08-11 09:27:06'], ['2013-08-11 09:19:06', '2013-08-11 09:30:06']]], '144': [[['2013-11-22 06:09:09', '2013-11-22 06:26:09'], ['2013-11-22 06:19:09', '2013-11...",app_app,False,1stFloor
307,Load_StatusBR2Lights>Load_StatusBR3Lights,0.991781,1.0,"{'41': [[['2013-08-11 18:19:05', '2013-08-12 00:00:04'], ['2013-08-11 18:29:05', '2013-08-11 20:29:05']]], '144': [[['2013-11-22 06:39:09', '2013-11-22 06:59:09'], ['2013-11-22 06:49:09', '2013-11...",app_app,False,2ndFloor
502,Load_StatusApplianceCooktop>Load_StatusApplianceRangeHood,0.986301,1.0,"{'41': [[['2013-08-11 08:14:06', '2013-08-11 08:31:06'], ['2013-08-11 08:14:06', '2013-08-11 08:31:06']], [['2013-08-11 12:01:05', '2013-08-11 12:18:05'], ['2013-08-11 12:01:05', '2013-08-11 12:18...",app_app,False,1stFloor
609,Load_StatusPlugLoadCanOpener>Load_StatusPlugLoadHandMixer,0.989041,1.0,"{'41': [[['2013-08-11 18:39:05', '2013-08-11 18:46:05'], ['2013-08-11 18:39:05', '2013-08-11 18:41:05']]], '144': [[['2013-11-22 18:39:08', '2013-11-22 18:46:08'], ['2013-11-22 18:39:08', '2013-11...",app_app,False,1stFloor


## Power consumption (Compute appliance power consumption need)

## Standby power consumption (Analyse if appliance have standby and how much power it consumes on standby)

## Energy needed: Power consumption multiply with minutes turned on / seconds per minute ((W/s)/(60s/min))*min

# Phase 3
Maybe the patterns can help some of the constraints
    o	Washing machine can be used at every hour, but maybe the constraints of the user can be found in the patterns
        -	E.g. General constraint that the appliances cant be started when the users are not home
        -	Dryer have to be started after the clotheswasher, so the clotheswasher cannot be started right before they go out of the house.